## Code Setup

In [ ]:
# If True, assumes everything is running locally.
IS_LOCAL = False

# Path to main directory
REMOTE_PATH = "/content/gdrive/Shareddrives/Birds and CS/Data/CA-paper"
LOCAL_PATH = "data/CA-EcoScape-Paper"
DATA_PATH = LOCAL_PATH if IS_LOCAL else REMOTE_PATH

In [ ]:
import os
import sys

Below, we install the packages from their git repositories, but you might want to change the code below into simply:

    !pip install ecoscape-connectivity
    !pip install ecoscape-utils

and use the standard packages.

In [ ]:
# Installs required packages
if not IS_LOCAL:
    !pip install git+https://github.com/ecoscape-earth/ecoscape-connectivity.git
    !pip install git+https://github.com/ecoscape-earth/ecoscape-utils.git

In [ ]:
# Connecting to Drive.
if not IS_LOCAL:
    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)

In [ ]:
import time
from ecoscape_utilities import BirdRun
from ecoscape_connectivity import compute_connectivity
from ecoscape_connectivity.util import read_transmission_csv

## Bird Run Definition

In [ ]:
bird_run = BirdRun(DATA_PATH)

birds = []

RUN_NAME = "test"
NUM_SIMULATIONS = 400
GAP_LENGTHS = [1, 2, 3, 4, 5, 6]
NUM_GAPS = [2, 4, 6, 8, 10, 15, 20, 30, 40]
COMPUTE_FLOW = True

for gap_length in GAP_LENGTHS:
    for num_gaps in NUM_GAPS:
        birds.append(bird_run.get_bird_run(
            "acowoo", "Acorn Woodpecker",
            do_gradient=COMPUTE_FLOW,
            run_name=RUN_NAME,
            hop_distance=gap_length, num_spreads=num_gaps,
            num_simulations=2 if IS_LOCAL else NUM_SIMULATIONS))

for gap_length in GAP_LENGTHS:
    for num_gaps in NUM_GAPS:
        birds.append(bird_run.get_bird_run(
            "oaktit308", "Oak Titmouse",
            do_gradient=COMPUTE_FLOW,
            run_name=RUN_NAME,
            hop_distance=gap_length, num_spreads=num_gaps,
            num_simulations=2 if IS_LOCAL else NUM_SIMULATIONS))

for gap_length in GAP_LENGTHS:
    for num_gaps in NUM_GAPS:
        birds.append(bird_run.get_bird_run(
            "stejay", "Steller's Jay",
            do_gradient=COMPUTE_FLOW,
            run_name=RUN_NAME,
            hop_distance=gap_length, num_spreads=num_gaps,
            num_simulations=2 if IS_LOCAL else NUM_SIMULATIONS))


In [ ]:
# Let's do the bird runs.
for bird in birds:

    # Creates output folder, if missing.
    bird_run.createdir_for_file(bird.repopulation_fn)
    bird_run.createdir_for_file(bird.gradient_fn)

    # Reads and translates the resistance dictionary.
    transmission_d = read_transmission_csv(bird.transmission_fn)

    # Creates output folder, if missing.
    bird_run.createdir_for_file(bird.repopulation_fn)
    bird_run.createdir_for_file(bird.log_fn)

    compute_connectivity(
        habitat_fn=bird.habitat_fn,
        terrain_fn=bird.terrain_fn,
        connectivity_fn=bird.repopulation_fn,
        flow_fn=bird.gradient_fn if bird.do_gradient else None,
        permeability_dict=transmission_d,
        gap_crossing=bird.hop_distance,
        num_gaps=bird.num_spreads,
        num_simulations=bird.num_simulations,
        seed_density=4,
        single_tile=False,
        tile_size=1024,
        tile_border=256,
    )

    print("Processed", bird.name, "hop", bird.hop_distance, "num", bird.num_spreads, "sim", bird.num_simulations)
